In [ ]:
! pip install azure-ai-vision-imageanalysis==1.0.0b1

In [1]:
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

In [13]:
import os

client = ImageAnalysisClient(
    #endpoint=os.environ["ENDPOINT"],
    #credential=AzureKeyCredential(os.environ["KEY"])
    endpoint="https://awf-custom-vision-001.cognitiveservices.azure.com/",
    credential=AzureKeyCredential("204d4cefaf0c4134a9a31299c12d5d85")
)

result = client.analyze(
    image_url="https://www.silicon.fr/wp-content/uploads/2023/03/Bill-Gates-Headshot-1021x683-1-684x513.jpg",
    visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ],
    gender_neutral_caption=True,
    language="en",
)

In [14]:
print(result)

{'modelVersion': '2023-10-01', 'captionResult': {'text': 'a person wearing glasses and a sweater', 'confidence': 0.7628260850906372}, 'metadata': {'width': 684, 'height': 513}, 'readResult': {'blocks': []}}


In [17]:
#!pip install pillow

In [21]:
import json
import os
from PIL import Image
from datetime import datetime

def create_coco_file(image_path, annotations, output_file):
    # Load image and get dimensions
    image = Image.open(image_path)
    width, height = image.size

    # Prepare COCO data structure
    coco_data = {
        "info": {
            "description": "COCO Dataset",
            "url": "http://cocodataset.org",
            "version": "1.0",
            "year": datetime.now().year,
            "contributor": "",
            "date_created": datetime.now().strftime("%Y/%m/%d")
        },
        "licenses": [
            {
                "id": 1,
                "name": "Unknown",
                "url": "http://example.com/license"
            }
        ],
        "images": [
            {
                "id": 1,
                "width": width,
                "height": height,
                "file_name": os.path.basename(image_path),
                "license": 1,
                "flickr_url": "",
                "coco_url": "",
                "date_captured": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
        ],
        "annotations": [],
        "categories": []
    }

    category_ids = {}
    annotation_id = 1

    # Process annotations
    for annotation in annotations:
        category_name = annotation['category']
        if category_name not in category_ids:
            category_id = len(category_ids) + 1
            category_ids[category_name] = category_id
            coco_data["categories"].append({
                "id": category_id,
                "name": category_name,
                "supercategory": "none"
            })

        category_id = category_ids[category_name]
        bbox = annotation['bbox']
        
        # COCO bounding box format is [x, y, width, height]
        coco_annotation = {
            "id": annotation_id,
            "image_id": 1,
            "category_id": category_id,
            "bbox": bbox,
            "area": bbox[2] * bbox[3],  # width * height
            "segmentation": [],
            "iscrowd": 0
        }
        coco_data["annotations"].append(coco_annotation)
        annotation_id += 1

    # Save to JSON file
    with open(output_file, 'w') as f:
        json.dump(coco_data, f, indent=4)

# Example usage
image_path = 'images/person.jpg'
annotations = [
    {
        "category": "cat",
        "bbox": [100, 150, 50, 50]  # [x, y, width, height]
    },
    {
        "category": "dog",
        "bbox": [200, 250, 75, 75]
    }
]
output_file = './results/coco_annotations.json'

#create_coco_file(image_path, annotations, output_file)



In [22]:
#create_coco_file("./images/building.jpg", annotations,  "./results/co")
create_coco_file(image_path, annotations, output_file)